In [1]:
import json
data = []
with open('../datasets/xquad/xquad.zh.json') as file:
    data_in_file = json.load(file)
for paragraphs in data_in_file['data']:
    for paragraph in paragraphs['paragraphs']:
        # articles.append(paragraph['context'])
        for qa in paragraph['qas']:
            data.append([paragraph['context'], qa['question'], [qa['answers'][0]['answer_start'], qa['answers'][0]['answer_start']+len(qa['answers'][0]['text'])]])

In [2]:
import public_params as pp
tokenizer = pp.tokenizer
data = [[tokenizer.convert_tokens_to_ids(list(each[0])), tokenizer.convert_tokens_to_ids(list(each[1])), each[2]] for each in data]

d:\install\Anaconda\envs\py3713\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def cut(data):

    q = [each[1] for each in data]
    c = [each[0] for each in data]
    labels = [each[2] for each in data]

    result = []
    maxlen4c = 330
    overlap = 20
    for i in range(len(data)):
        maxlen4c = 505 - len(q[i])
        if len(c[i]) > maxlen4c:
            for j in range(0, len(c[i])-overlap, maxlen4c-overlap):
                if labels[i][0]<j and labels[i][1]<j:
                    result.append([c[i][j: j+maxlen4c], q[i], [0, 0]])
                elif labels[i][0]<j and labels[i][1]>=j:
                    result.append([c[i][j: j+maxlen4c], q[i], [0, labels[i][1]-(j)]])
                elif labels[i][0]>=j and labels[i][1]<j+maxlen4c:
                    result.append([c[i][j: j+maxlen4c], q[i], [labels[i][0]-(j), labels[i][1]-(j)]])
                elif labels[i][0]<j+maxlen4c and labels[i][1]>=j+maxlen4c:
                    result.append([c[i][j: j+maxlen4c], q[i], [labels[i][0]-(j), maxlen4c-1]])
                elif labels[i][0]>=j+maxlen4c and labels[i][1]>=j+maxlen4c:
                    result.append([c[i][j: j+maxlen4c], q[i], [0, 0]])
                if j+maxlen4c >= len(c[i]):
                    break
        else: 
            result.append([c[i], q[i], labels[i]])
    return result

data = cut(data)

In [4]:
import numpy
numpy.save(f'./data4finetuning/xquad', data)

d:\install\Anaconda\envs\py3713\lib\site-packages\numpy\lib\npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


In [5]:
ml = max([len(each[0])+len(each[1]) for each in data])
ml

505